In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lh_sampling.sampler import lhs
from lh_sampling.util import inv_cdf, ecdf
from scipy.stats import norm, multivariate_normal
from lh_sampling import get_data

In [ ]:
def str2float(val):
    if val == '':
        return np.nan
    else:
        return float(val)

def str2cat(val):
    if val.find('CAL') != -1:
        return 1
    return 0
        
df = pd.read_csv(get_data('/data/SR2021-12 Spreadsheet_IVESPA.csv'), usecols=[1,4,6,7,8,9,11,12,13],
                 converters={6:str2float, 7:str2cat, 8:str2float,
                             9:str2float, 11:str2cat, 12:str2float,
                             13:str2cat})

In [ ]:
df.rename(columns={'MER_kg/s':'MER', 'Vent_elevation_km':'Vent_elevation',
                   'Column_height_km':'Column_height', 'Duration_hr':'Duration',
                   'Magma_type':'Magma type'},
          inplace=True)

# remove calculated values
df['Column_height'] = np.where(df.Column_height_note==1, np.nan, df['Column_height'])
df['MER'] = np.where(df.MER_note==1, np.nan, df['MER'])
df['Duration'] = np.where(df.Duration_note==1, np.nan, df['Duration'])

# Add log columns
df['Volume'] = df['MER'] * df['Duration']
df['log Volume'] = np.log(df['Volume'])
df['log MER'] = np.log(df['MER'])
df['log Column height'] = np.log(df['Column_height'])
df['log Duration'] = np.log(df['Duration'])

In [ ]:
# Interactive scatter matrix plot
fig = px.scatter_matrix(df, dimensions=['Column_height', 'MER', 'Duration', 'Volume'], color="Magma type")
fig.update_layout(width=1000, height=1000)

In [ ]:
g = sns.PairGrid(df, vars=['log Duration', 'log MER', 'log Column height'], hue='Magma type',
                 diag_sharey=False, height=3)
g.map_upper(sns.scatterplot, s=15)
g.map_lower(sns.kdeplot)
g.map_diag(sns.ecdfplot, lw=2)
g.add_legend()
g.savefig('scatter_matrix.png')

In [ ]:
df_m = df[['log Column height', 'log MER', 'log Duration']].where(df['Magma type'] == 'Mafic')
df_i = df[['log Column height', 'log MER', 'log Duration']].where(df['Magma type'] == 'Intermediate')
df_tmp = df_m
lsmp = lhs(3, 20, centered=True)
U = np.linalg.cholesky(df_tmp.cov().values)
mean = df_tmp.mean()

if False:
    def resample(df, key, samples):
        x = df[key].dropna().values
        x_sorted, cdf = ecdf(x)
        x_new = inv_cdf(x, samples)
        return x_new

    lh_samples_ch = resample(df_tmp, 'log Column height', lsmp[:,0])
    lh_samples_mer = resample(df_tmp, 'log MER', lsmp[:,1])
    lh_samples_d = resample(df_tmp, 'log Duration', lsmp[:,2])
    lh_samples = np.vstack((lh_samples_ch, lh_samples_mer, lh_samples_d))

if True:
    lh_samples_x = norm.ppf(lsmp[:, 0], loc=0, scale=1)
    lh_samples_y = norm.ppf(lsmp[:, 1], loc=0, scale=1)
    lh_samples_z = norm.ppf(lsmp[:, 2], loc=0, scale=1)
    lh_samples = np.vstack((lh_samples_x, lh_samples_y, lh_samples_z))
    lh_samples = np.dot(U, lh_samples)
    lh_samples = lh_samples+mean.values[:, np.newaxis]
    
dfr = pd.DataFrame(lh_samples.T, columns=df_tmp.columns)
dfr['Category'] = ['resampled']*dfr.shape[0]
df_tmp['Category'] = ['original']*df_tmp.shape[0]
dfr = pd.concat((df_tmp, dfr)).reset_index()

In [ ]:
g = sns.PairGrid(dfr, vars=['log Duration', 'log MER', 'log Column height'], hue='Category',
                 diag_sharey=False, height=3)
g.map_upper(sns.scatterplot, s=15)
g.map_lower(sns.kdeplot)
g.map_diag(sns.ecdfplot, lw=2)
g.add_legend()
g.savefig('scatter_matrix_resample.png')

In [ ]:
np.exp(2)

In [ ]:
dfr[dfr.Category == 'original'].cov()

In [ ]:
dfr[dfr.Category == 'resampled'].cov()

## Etna analysis

In [ ]:
df_etna = df[df['Volcano']=='Etna']
df_etna = df_etna.sort_values(axis=0, by='MER')

In [ ]:
s = sns.pairplot(df_etna, vars=['Duration', 'MER', 'Column_height'], hue='Magma type')
for i in range(3):
    for j in range(3):
        s.axes[i,j].set(xscale="log", yscale="log")

In [ ]:
s = sns.displot(df_etna, x="MER", kind="kde")
s.set(xscale="log", yscale="log")